In [16]:
from requests_html import HTMLSession
from requests import get
# import scrapy
# from scrapy.crawler import CrawlerProcess
from random import choice
# import pandas as pd
# import sqlite3
from pprint import pprint as print
from time import sleep

In [5]:
def data_next_page_id(base_size=60, maximum_size=110):
    count = 1
    session = HTMLSession()
    next_page_ids = [0]
    while count != 0:
        new_next_page_ids = []
        for _ in next_page_ids:
            url = f'https://api.divar.ir/v8/map-discovery/bbox/posts?lon1=50.780457955008785&lat1=35.43862123757947&lon2=51.702507060611595&lat2=35.94376848564657&nextPageId={_}&filters=building-age=0-10&filters=category=apartment-sell&filters=size={base_size}-{maximum_size}'
            response = session.get(url)
            new_next_page_ids.append(response.json().get("next_page_id"))
        next_page_ids.extend(new_next_page_ids)  # Use extend() instead of append()
        count -= 1
    return list(set(next_page_ids))

In [8]:
apartment = data_next_page_id(60,110)
def tokens():
    urls=[]
    tokens=[]
    for _ in apartment:
        urls.append(f'https://api.divar.ir/v8/map-discovery/bbox/posts?lon1=50.780457955008785&lat1=35.43862123757947&lon2=51.702507060611595&lat2=35.94376848564657&nextPageId={_}&filters=building-age=0-10&filters=category=apartment-sell&filters=size=60-110')
    for url in urls:
        session = HTMLSession()
        response=session.get(url)
        datas=response.json()["posts"]
        for _ in datas:
            tokens.append(_.get("token"))
    return tokens
# tokens()

In [18]:
def r_html():
    for url in tokens():
        # session=HTMLSession()
        user_agent=choice([
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/115.0",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 13.4; rv:109.0) Gecko/20100101 Firefox/115.0",
            "Mozilla/5.0 (X11; Linux i686; rv:109.0) Gecko/20100101 Firefox/115.0",
            "Mozilla/5.0 (iPhone; CPU iPhone OS 13_4_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) FxiOS/115.0 Mobile/15E148 Safari/605.1.15",
            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.360",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.5 Safari/605.1.15",
            'Mozilla/5.0 (Windows NT 10.0; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/114.0.1823.86',
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 OPR/100.0.0.0",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Vivaldi/6.1.3035.111",

        ])
        headers={'User-Agent':user_agent}
        response=get(f"https://divar.ir/v/{url}",headers=headers)
        sleep(0.05)
        # response.html.render()
    print(response.text)
r_html()

('<!DOCTYPE html>\n'
 '<html>\n'
 '\n'
 '<head>\n'
 '    <title>Too many requests</title>\n'
 '</head>\n'
 '\n'
 '<body>\n'
 '    <h2>429 Too many requests</h2>\n'
 '    <div>You can have 15 requests every 15 seconds</div>\n'
 '</body>\n'
 '\n'
 '</html>')


class Divar(scrapy.Spider):
    name="divar"
    start_urls=[f"https://divar.ir/v/{_}" for _ in tokens()]

    def parse(self, response):
        location= str(response.xpath('//*[@id="app"]/div[2]/div/div[1]/div[1]/div[1]/div[2]/div[2]/text()').extract()).split("در")[1]
        price_per_meter=str(response.xpath("//*[@id='app']/div[2]/div/div[1]/div[1]/div[4]/div[3]/div[2]/text()").extract())
        print(5*"***")
        print(price_per_meter)

if __name__=="__main__":
    p=CrawlerProcess()
    p.crawl(Divar)
    p.start()